In [1]:
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
from pyspark.sql import functions as F
import sys
from hops import hdfs as hdfs
print(hdfs.project_path())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
18,application_1613204927129_0025,pyspark,idle,Link,Link


SparkSession available as 'spark'.
hdfs://rpc.namenode.service.consul:8020/Projects/pittre/

In [2]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/5-10-2-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show()

+---+----+-----+
| id|  ts|   f1|
+---+----+-----+
|  1|1002| f2-1|
|  1|1004| f2-2|
|  1|1006| f2-3|
|  1|1008| f2-4|
|  1|1010| f2-5|
|  2|1002| f2-6|
|  2|1004| f2-7|
|  2|1006| f2-8|
|  2|1008| f2-9|
|  2|1010|f2-10|
|  3|1002|f2-11|
|  3|1004|f2-12|
|  3|1006|f2-13|
|  3|1008|f2-14|
|  3|1010|f2-15|
|  4|1002|f2-16|
|  4|1004|f2-17|
|  4|1006|f2-18|
|  4|1008|f2-19|
|  4|1010|f2-20|
+---+----+-----+
only showing top 20 rows

In [3]:
fg2_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f2", StringType(), True)    
])

fg2=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/5-10-3-out.csv", header=True, schema=fg2_schema)
fg2=fg2.select([col("id").alias("id_2"), col("ts").alias("ts_2"), col("f2")])
fg2.show()

+----+----+-----+
|id_2|ts_2|   f2|
+----+----+-----+
|   1|1003| f3-1|
|   1|1006| f3-2|
|   1|1009| f3-3|
|   2|1003| f3-4|
|   2|1006| f3-5|
|   2|1009| f3-6|
|   3|1003| f3-7|
|   3|1006| f3-8|
|   3|1009| f3-9|
|   4|1003|f3-10|
|   4|1006|f3-11|
|   4|1009|f3-12|
|   5|1003|f3-13|
|   5|1006|f3-14|
|   5|1009|f3-15|
+----+----+-----+

In [4]:
from pyspark.sql import Window
from pyspark.sql.functions import lit, when, col, lag, rank

win = Window.partitionBy(["id", "ts"]).orderBy(col('ts_2').desc())

In [7]:
final=fg1.join(fg2, (fg1.id == fg2.id_2) & (fg1.ts >= fg2.ts_2)) \
    .withColumn("id_rank", rank().over(win)) \
    .filter(col("id_rank") == 1).drop(col("id_rank")).drop(col("ts_2")).drop(col("id_2"))

In [8]:
final.show()

+---+----+-----+-----+
| id|  ts|   f1|   f2|
+---+----+-----+-----+
|  1|1004| f2-2| f3-1|
|  1|1006| f2-3| f3-2|
|  1|1008| f2-4| f3-2|
|  1|1010| f2-5| f3-3|
|  2|1004| f2-7| f3-4|
|  2|1006| f2-8| f3-5|
|  2|1008| f2-9| f3-5|
|  2|1010|f2-10| f3-6|
|  3|1004|f2-12| f3-7|
|  3|1006|f2-13| f3-8|
|  3|1008|f2-14| f3-8|
|  3|1010|f2-15| f3-9|
|  4|1004|f2-17|f3-10|
|  4|1006|f2-18|f3-11|
|  4|1008|f2-19|f3-11|
|  4|1010|f2-20|f3-12|
|  5|1004|f2-22|f3-13|
|  5|1006|f2-23|f3-14|
|  5|1008|f2-24|f3-14|
|  5|1010|f2-25|f3-15|
+---+----+-----+-----+